# Check Conda Env Name

In [1]:
import sys
sys.executable.split('/')[-3]

'pytor'

# Result Filename

In [2]:
result_filename='Results_vox1.csv'

# Imports

In [3]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.face import FaceClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from array import array
import os
import sys
import time
from PIL import Image
import csv
from time import sleep
from glob import glob

## API Setting 

In [4]:
subscription_key = "4d673dca4fbb43ab8906a7ae5691c866"
endpoint="https://advdf.cognitiveservices.azure.com/"
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
subscription_key = "2b4d12d9d44940bbbd8c48c8f3c0e575"
endpoint="https://advdf-similarity.cognitiveservices.azure.com/"
face_client=FaceClient(endpoint, CognitiveServicesCredentials(subscription_key))

# VoxCeleb 1

### Describe an Image

In [5]:
'''
Describe an Image - local
This example describes the contents of an image with the confidence score.'Scarlett Johansson_fake.png'
'''
def describe_image(filename):
#     print("===== Describe an image - local =====")
    # Call API
    image= open(filename,"rb")
    description_results = computervision_client.describe_image_in_stream(image)

    # Get the captions (descriptions) from the response, with confidence level
#     print("Description of local image: ")
    if (len(description_results.captions) == 0):
#         print("No description detected.")
        return "No description detected."
    else:
        for caption in description_results.captions:
#             print("'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100))
            return "'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100)

### Detect Domain-specific Content

In [6]:
'''
Detect Domain-specific Content - local
This example detects celebrites and landmarks in local images.
'''
def domain_specific_content(filename):
#     print("===== Detect Domain-specific Content - local =====")
    # Image of one or more celebrities
    image= open(filename,"rb")
    # image = image.read('Scarlett.png')

    # Call API with content type (celebrities) and URL
    detect_domain_results_celebs_remote = computervision_client.analyze_image_by_domain_in_stream("celebrities", image)
    # Print detection results with name
#     print("Celebrities in the local image:")
    if len(detect_domain_results_celebs_remote.result["celebrities"]) == 0:
        print("No celebrities detected.")
        return "",""
    else:
        for celeb in detect_domain_results_celebs_remote.result["celebrities"]:
#             print(celeb["name"])
#             print(celeb["confidence"])
            return celeb["name"],celeb["confidence"]

In [7]:
def get_name(filename):
    return os.path.splitext(os.path.basename(filename))[0].split("_")[0]

In [8]:
def check_similarity(src_image, tar_image):
    # Detect a face in an image that contains a single face
    src_image_name = os.path.basename(src_image)
    detected_faces = face_client.face.detect_with_stream(open(src_image,"rb"))
    if not detected_faces:
        print('No face detected from image {}'.format(single_image_name))
        return 0
#         raise Exception('No face detected from image {}'.format(single_image_name))

    # Display the detected face ID in the first single-face image.
    # Face IDs are used for comparison to faces (their IDs) detected in other images.
#     print('Detected face ID from', single_image_name, ':')
#     for face in detected_faces: 
#         print (face.face_id)
#     print()

    # Save this ID for use in Find Similar
    first_image_face_ID = detected_faces[0].face_id
    
    multi_image_name = os.path.basename(tar_image)
    detected_faces2 = face_client.face.detect_with_stream(open(tar_image,"rb"))
    if not detected_faces2:
        print('No face detected from image {}'.format(multi_image_name))
        return 0
#         raise Exception('No face detected from image {}'.format(multi_image_name))
    
    # Search through faces detected in group image for the single face from first image.
    # First, create a list of the face IDs found in the second image.
    second_image_face_IDs = list(map(lambda x: x.face_id, detected_faces2))
    # Next, find similar face IDs like the one detected in the first image.
    similar_faces = face_client.face.find_similar(face_id=first_image_face_ID, face_ids=second_image_face_IDs)
    
    if not similar_faces:
        print('No similar faces found in', multi_image_name, '.')
        return 0
    else:
        # Print the details of the similar faces detected
#         print('Similar faces found in', multi_image_name + ':')
        for face in similar_faces:
        #     print(face)
            return face.confidence
#             print('  Confidence:', face.confidence)
#             first_image_face_ID = face.face_id
#             # The similar face IDs of the single face image and the group image do not need to match, 
#             # they are only used for identification purposes in each image.
#             # The similar faces are matched using the Cognitive Services algorithm in find_similar().
#             face_info = next(x for x in detected_faces2 if x.face_id == first_image_face_ID)
#             if face_info:
#                 print('  Face ID: ', first_image_face_ID)
#                 print('  Face rectangle:')
#                 print('    Left: ', str(face_info.face_rectangle.left))
#                 print('    Top: ', str(face_info.face_rectangle.top))
#                 print('    Width: ', str(face_info.face_rectangle.width))
#                 print('    Height: ', str(face_info.face_rectangle.height))

In [10]:
file=open(result_filename, 'w', newline='')
writer = csv.writer(file)
writer.writerow(["Filename", "Src","Tar", "Src Pred.", "Src Conf.", "Tar Pred.","Tar Conf.", "Similarity","Src Des", "Tar Des"])
file.close()
actors_list=glob("/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/*")
count=0
for actor in actors_list:
    count+=1
    print(os.path.basename(actor))
    real_image=os.path.join("/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/real",
                                os.path.basename(actor)+".jpg")
    fake_images_list=glob(os.path.join(actor,"*/*/*"))
    print(real_image)
    result_real_pred,result_real_conf=domain_specific_content(real_image)
    real_des=describe_image(real_image)
    src=get_name(real_image)
    for fake_image in fake_images_list:
        print(fake_image)
#         continue
        result_fake_pred,result_fake_conf=domain_specific_content(fake_image)
        fake_des=describe_image(fake_image)
        tar=get_name(fake_image)
        similarity=check_similarity(real_image,fake_image)
#             print(fake_image,src,tar,
#                   result_real_pred,result_real_conf,
#                   result_fake_pred,result_fake_conf,
#                   similarity,real_des,fake_des)
        file=open(result_filename, 'a', newline='')
        writer = csv.writer(file)
        writer.writerow([fake_image,src,tar,
              result_real_pred,result_real_conf,
              result_fake_pred,result_fake_conf,
              similarity,real_des,fake_des])
        file.close()
#             sleep(10)
#             break
#         break
#     break
# file.close()
print("Total:",count)

Paul Scheer
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/real/Paul Scheer.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Paul Scheer/BlPusL_Bv6k/1.6/0008000.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Paul Scheer/BlPusL_Bv6k/1.6/0006600.jpg
No celebrities detected.
No similar faces found in 0006600.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Paul Scheer/BlPusL_Bv6k/1.6/0006500.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Paul Scheer/BlPusL_Bv6k/1.6/0006625.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Paul Scheer/BlPusL_Bv6k/1.6/0004425.jpg
No celebrities detected.
No similar faces found in 0004425.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Paul Scheer/BlPusL_Bv6k/1.6/0007350.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Pa

No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Nicole Richie/0EwaRLkCnxI/1.6/0010550.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Nicole Richie/0EwaRLkCnxI/1.6/0009575.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Nicole Richie/0EwaRLkCnxI/1.6/0011425.jpg
No celebrities detected.
Mandy Patinkin
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/real/Mandy Patinkin.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Mandy Patinkin/PoK7A-G4uLY/1.6/0004900.jpg
No celebrities detected.
No similar faces found in 0004900.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Mandy Patinkin/PoK7A-G4uLY/1.6/0014075.jpg
No celebrities detected.
No similar faces found in 0014075.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Mandy Patinkin/PoK7A-G4uLY/1.6/0006725.jpg
No cel

/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Abigail Spencer/r4IZRsEGDFs/1.6/0031100.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Abigail Spencer/r4IZRsEGDFs/1.6/0033200.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Abigail Spencer/r4IZRsEGDFs/1.6/0032675.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Abigail Spencer/r4IZRsEGDFs/1.6/0045350.jpg
No similar faces found in 0045350.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Abigail Spencer/r4IZRsEGDFs/1.6/0032225.jpg
No similar faces found in 0032225.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Abigail Spencer/r4IZRsEGDFs/1.6/0019500.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Abigail Spencer/r4IZRsEGDFs/1.6/0041475.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung 

/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Chris Hemsworth/C4gZU4pvY3g/1.6/0003800.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Chris Hemsworth/C4gZU4pvY3g/1.6/0002050.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Chris Hemsworth/C4gZU4pvY3g/1.6/0001575.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Chris Hemsworth/C4gZU4pvY3g/1.6/0002975.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Chris Hemsworth/C4gZU4pvY3g/1.6/0001950.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Chris Hemsworth/C4gZU4pvY3g/1.6/0005075.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Chris Hemsworth/C4gZU4pvY3g/1.6/0002400.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Chris Hemsworth/C4gZU4pvY3g/1.6/0002025.jp

/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Rory Kennedy/Qq_ncaVFAMs/1.6/0052400.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Rory Kennedy/Qq_ncaVFAMs/1.6/0028625.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Rory Kennedy/Qq_ncaVFAMs/1.6/0050200.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Rory Kennedy/Qq_ncaVFAMs/1.6/0042875.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Rory Kennedy/Qq_ncaVFAMs/1.6/0086000.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Rory Kennedy/Qq_ncaVFAMs/1.6/0019600.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Rory Kennedy/Qq_ncaVFAMs/1.6/0049050.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Rory Kennedy/Qq_ncaVFAMs/1.6/0043650.jpg
/home/sha/project/adve

...

/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Julian Sands/j3x790iQEmg/1.6/0004000.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Julian Sands/j3x790iQEmg/1.6/0003875.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Julian Sands/j3x790iQEmg/1.6/0003850.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Julian Sands/j3x790iQEmg/1.6/0002775.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Julian Sands/j3x790iQEmg/1.6/0002975.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Julian Sands/j3x790iQEmg/1.6/0005500.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Julian Sands/j3x790iQEmg/1.6/0003000.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Julian Sands/j3x790iQEmg/1.6/0004750.jpg
/home/sha/project/adve

No similar faces found in 0005100.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Nathaniel Buzolic/iH5Sk0y4r8Y/1.6/0007125.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Nathaniel Buzolic/iH5Sk0y4r8Y/1.6/0005225.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Nathaniel Buzolic/iH5Sk0y4r8Y/1.6/0008975.jpg
No similar faces found in 0008975.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Nathaniel Buzolic/iH5Sk0y4r8Y/1.6/0003875.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Nathaniel Buzolic/iH5Sk0y4r8Y/1.6/0003800.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Nathaniel Buzolic/iH5Sk0y4r8Y/1.6/0008625.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Nathaniel Buzolic/iH5Sk0y4r8Y/1.6/0007775.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fak

/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Todd Lasance/_9x1vLInPc0/1.6/0002175.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Todd Lasance/_9x1vLInPc0/1.6/0002475.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Todd Lasance/_9x1vLInPc0/1.6/0002200.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Todd Lasance/_9x1vLInPc0/1.6/0002575.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Todd Lasance/_9x1vLInPc0/1.6/0003800.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Todd Lasance/_9x1vLInPc0/1.6/0004550.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Todd Lasance/_9x1vLInPc0/1.6/0001600.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Todd Lasance/_9x1vLInPc0/1.6/0001575.jpg
/home/sha/project/adversarial_deepfake/src/Simi

No similar faces found in 0007700.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Brenda Blethyn/q2BoSBDt3qo/1.6/0013125.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Brenda Blethyn/q2BoSBDt3qo/1.6/0003375.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Brenda Blethyn/q2BoSBDt3qo/1.6/0003275.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Brenda Blethyn/q2BoSBDt3qo/1.6/0016825.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Brenda Blethyn/q2BoSBDt3qo/1.6/0014700.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Brenda Blethyn/q2BoSBDt3qo/1.6/0014600.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Brenda Blethyn/q2BoSBDt3qo/1.6/0002025.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Brenda Blethyn/q2BoSBDt3qo/1.6/0019050.jpg
/home/sha/projec

No similar faces found in 0008400.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Thomas Jane/5fUxYQUBtLA/1.6/0008225.jpg
No celebrities detected.
No similar faces found in 0008225.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Thomas Jane/5fUxYQUBtLA/1.6/0003650.jpg
No similar faces found in 0003650.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Thomas Jane/5fUxYQUBtLA/1.6/0006850.jpg
No celebrities detected.
No similar faces found in 0006850.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Thomas Jane/5fUxYQUBtLA/1.6/0008375.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Thomas Jane/5fUxYQUBtLA/1.6/0006525.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Thomas Jane/5fUxYQUBtLA/1.6/0006275.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/Samsung TH Vox1/fake/Thomas Jane/5fUxYQUBtLA/1